In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
import scipy
%matplotlib inline

In [2]:
initial_cwd = os.getcwd()
initial_cwd

'D:\\projects\\kr\\vzam\\notebooks'

In [3]:
print(initial_cwd)
os.chdir(initial_cwd)
os.chdir('..')
print(os.getcwd())

D:\projects\kr\vzam\notebooks
D:\projects\kr\vzam


In [4]:
from vzam import *

In [44]:
RESIZE = (200,200)
SOURCE_FRAMERATE = 5
QUERY_FRAMERATE = 5
TIRI_BUFFER_SIZE = 5
HASH_SIZE = 100

In [6]:
TMP_DIR = 'tmp'
FRAMES_DIR = 'tmp/frames'
DATA_DIR = 'data'
if not os.path.exists(TMP_DIR):
    os.mkdir(TMP_DIR)
if not os.path.exists(FRAMES_DIR):
    os.mkdir(FRAMES_DIR)
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

In [78]:
FRAMES_DIR = None

In [7]:
VCDB_CSV = 'data/vcdb.csv'
PATH_PREFIX = '../vcdb/'

In [8]:
vcdb_df = pd.read_csv(VCDB_CSV)
vcdb_df.head()

,query,source,source_id
0,core_dataset/baggio_penalty_1994\3504e360accba...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,core_dataset/baggio_penalty_1994\37b31d607d31a...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,core_dataset/baggio_penalty_1994\458db5aa227ae...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,core_dataset/baggio_penalty_1994\5c5714c0a56fd...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,core_dataset/baggio_penalty_1994\67824b87c0698...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [9]:
query_source_lut = vcdb_df[['query', 'source_id']]
query_source_lut['query_id'] = query_source_lut['query'].apply(lambda x: os.path.basename(x).split('\\')[-1])
query_source_lut = query_source_lut[['query_id', 'source_id']]
query_source_lut.head()

,query_id,source_id
0,3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,37b31d607d31a47d347b15dae2b8aa63e57861eb.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,458db5aa227ae49ceb8bc1bed5f9cf5b4bed63f2.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,5c5714c0a56fd2a96f99db2f59b0d03659d77cdf.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,67824b87c0698c499acad123b7498ae17f97bf6d.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [10]:
query_source_lut.to_csv('data/vcdb_query_source_lut.csv', index=False)

In [11]:
vcdb_df.tail()

,query,source,source_id
514,core_dataset/zidane_headbutt\afc75e8c02a5ecba4...,NaN,NaN
515,core_dataset/zidane_headbutt\b91aa7020c5dd0a65...,NaN,NaN
516,core_dataset/zidane_headbutt\cb593efe5b194fe67...,NaN,NaN
517,core_dataset/zidane_headbutt\cc3b8791039b9e7f6...,NaN,NaN
518,core_dataset/zidane_headbutt\e2d1ca4b2657cd820...,NaN,NaN


In [12]:
source_paths = vcdb_df.dropna().source.unique()

In [90]:
source_video_paths = [os.path.join(PATH_PREFIX, fpath) for fpath in source_paths]
source_video_paths

['../vcdb/core_dataset/baggio_penalty_1994\\bb604f57a18455867544e79c2e32bf5583c358d4.flv',
 '../vcdb/core_dataset/beautiful_mind_game_theory\\46f2e964ae16f5c27fad70d6849c76616fad7502.flv',
 '../vcdb/core_dataset/beckham_70_yard_goal\\97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv',
 '../vcdb/core_dataset/bill_clinton_apology_speech\\410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv',
 '../vcdb/core_dataset/bolt_beijing_100m\\098df9c477ecf6c4f3818b198490b0258163015d.flv',
 '../vcdb/core_dataset/brazil_vs_brazil_nike_commercial_2012\\3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv',
 '../vcdb/core_dataset/david_beckham_lights_the_olympic_torch\\d2015b438b70f022967713d6f977ebc67a16839e.flv',
 '../vcdb/core_dataset/dove_evolution_commercial\\127dab55025984673f65d3a23b1fea99ecc79b15.mp4',
 '../vcdb/core_dataset/endless_love\\3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv']

In [91]:
assert os.path.exists(source_video_paths[0])

In [92]:
source_processed_paths = [os.path.join(TMP_DIR, '_'+os.path.basename(fpath)) for fpath in source_video_paths]
source_processed_paths

['tmp\\_bb604f57a18455867544e79c2e32bf5583c358d4.flv',
 'tmp\\_46f2e964ae16f5c27fad70d6849c76616fad7502.flv',
 'tmp\\_97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv',
 'tmp\\_410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv',
 'tmp\\_098df9c477ecf6c4f3818b198490b0258163015d.flv',
 'tmp\\_3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv',
 'tmp\\_d2015b438b70f022967713d6f977ebc67a16839e.flv',
 'tmp\\_127dab55025984673f65d3a23b1fea99ecc79b15.mp4',
 'tmp\\_3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv']

In [93]:
def prepare_videos(in_paths, out_paths, delete_old=True):
    for in_path, out_path in zip(in_paths, out_paths):
        try:
            if delete_old and os.path.exists(out_path):
                os.remove(out_path)
            preprocess_video(in_path, out_path, resize=RESIZE, target_framerate=SOURCE_FRAMERATE)
        except Exception as e:
#             print(e)
            pass
        print(in_path, 'done')
    for path in out_paths:
        assert os.path.exists(path)

In [17]:
# prepare_videos(source_video_paths, source_processed_paths)

In [94]:
query_paths = vcdb_df['query']
query_video_paths = [os.path.join(PATH_PREFIX, fname) for fname in query_paths]
assert os.path.exists(query_video_paths[0])
query_processed_paths = [os.path.join(TMP_DIR, '_'+os.path.basename(fpath)) for fpath in query_video_paths]
len(query_processed_paths)

519

In [95]:
# prepare_videos(query_video_paths, query_processed_paths)

In [98]:
source_df = get_rhash_df(source_processed_paths, buffer_size=TIRI_BUFFER_SIZE, hash_size=HASH_SIZE, write_frames_dir=FRAMES_DIR)
source_df['id'] = source_df['id'].apply(lambda x: x.replace('_', ''))

tmp\_bb604f57a18455867544e79c2e32bf5583c358d4.flv
tmp\_46f2e964ae16f5c27fad70d6849c76616fad7502.flv
tmp\_97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv
tmp\_410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv
tmp\_098df9c477ecf6c4f3818b198490b0258163015d.flv
tmp\_3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv
tmp\_d2015b438b70f022967713d6f977ebc67a16839e.flv
tmp\_127dab55025984673f65d3a23b1fea99ecc79b15.mp4
tmp\_3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv


In [99]:
source_df.head()

,feature,ts,id
0,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, ...",0.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",0.95,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",1.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",1.95,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,"[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, ...",2.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [100]:
source_df.shape

(2677, 3)

In [87]:
source_df.to_csv(DATA_DIR+'/vcdb_rhash_source.csv', index=False)

In [88]:
query_df = get_rhash_df(query_processed_paths, buffer_size=TIRI_BUFFER_SIZE, hash_size=HASH_SIZE, write_frames_dir=FRAMES_DIR)
query_df['id'] = query_df['id'].apply(lambda x: x.replace('_', ''))
query_df.head()

tmp\_3504e360accbaccb1580befbb441f1019664c2bb.mp4
tmp\_37b31d607d31a47d347b15dae2b8aa63e57861eb.flv
tmp\_458db5aa227ae49ceb8bc1bed5f9cf5b4bed63f2.flv
tmp\_5c5714c0a56fd2a96f99db2f59b0d03659d77cdf.flv
tmp\_67824b87c0698c499acad123b7498ae17f97bf6d.flv
tmp\_6d1466ebc4de7e5ddb229bde090b5c5acac15c0c.flv
tmp\_6d1a89c83d554fc6a5e39fcadb172a79baf140fd.mp4
tmp\_8084216caff6082b4e71ae4bbfe556f28a68485f.flv
tmp\_9423340949f55d7a089e881ccc35f23dff3fccf7.flv
tmp\_e901a631b00f4ad0c9d161d686fac1339e1e3535.flv
tmp\_5f6527456bf2b4c460737a1c222dbd0e70009030.flv
tmp\_6171d3d87ae377e497199554033bca96a263277b.mp4
tmp\_904c8ebf782357ae78ebd205fe3428ad76b975a5.flv
tmp\_b41c61fb756014444b7974ef25f2c76d5c3b42e4.flv
tmp\_0056649006c7a250cb4174f1e924768ba2c9d599.flv
tmp\_1357538fd1fd2975a8b3f750d5f6b13695aa4e28.flv
tmp\_1f4f6311bb4acc0bc72e4a915a32093a7a85741c.flv
tmp\_37a935b4c21662e183a0dd632f40d5fd6900c42b.mp4
tmp\_3fe2e54fdb267849aa8aeb0b39ec0d1c7075b391.flv
tmp\_4ffe88b2da93f727952dc3104e70c0b0665d9e1a.flv


tmp\_ce1451b329a9cd5ad178f0927644f81fad4c69c2.flv
tmp\_d31adc1a71b72bb36f32b67cb6d52d4840a322c5.flv
tmp\_de71afa5fe2363589c28021940d2902d8701efdf.flv
tmp\_df4001c1e9d3e8fe3c61d9b6bac46cd377e6d2be.flv
tmp\_df839544c199b12b5de023c1ee8511b947f5c999.flv
tmp\_e018528afcd7682556a4049b76d76105f60fa922.flv
tmp\_f5d187eb9c7e5cfc6eaa206f479ede5c10603c68.flv
tmp\_2ba013a3f6fa57793c242766e27ec77d001fe03a.flv
tmp\_38f11a4162d8e94227ac644f117f942735b9a504.mp4
tmp\_39c96fc7117a0ff1fdc02637285c13d8efe7ce93.flv
tmp\_7caa7fcef0e0b17725a0bda8b101d819e6ca2dbd.flv
tmp\_801dbd9a5a5f97442c7bbd14046d71e67cccbf4b.mp4
tmp\_9ac82ed37390f45bdc81b699c53085b29143ee70.flv
tmp\_be51784c0b0676bf487376af362b088922efa954.flv
tmp\_bf582249cfc79d691195a8681961029cc5149a76.flv
tmp\_d2f4aca96d44a144b12abc300b781b8ff8bb0842.flv
tmp\_e2adc784b83446ae775f698b9d17c9fd392b2f75.flv
tmp\_e8aa839caca7b98a0bf5437db144f23530d39b25.flv
tmp\_06f0dd6a00b38b553844d0550696e5ce1fb448b6.flv
tmp\_10732a0e6a0edef9dcbb2155236e46a7ed5047c0.flv


tmp\_8bb9090a95a07edf2fd021d64335bf7b4f428eb2.flv
tmp\_952cdd6aa1a8c2f63f9900960b3e18ffb217b97f.flv
tmp\_96642d7998cdd226b395153b7d8f3d36655c0176.flv
tmp\_9f3d3762943c662470d78368a29a067952a97933.flv
tmp\_a257de343d6734f0fc478da98cfedf773ea407a4.mp4
tmp\_aa314017a4a18f05208d8702c27d16d57df3f55f.mp4
tmp\_aba6851ca7f78f3a83216005bef58766e3dfb26d.flv
tmp\_bdce7faffa2ce63db64ddab4fa10ac6752079020.flv
tmp\_db06eed340d0873fc48b2129115f16c79c48678c.flv
tmp\_e7831eba03ee7083a9d547fdde6adadaa05f3eeb.flv
tmp\_010aeeb1a5d471418ceab8f8c0d5f34d84059cff.flv
tmp\_136eff8ef85eee42b2370ad6e444498980ba63a8.mp4
tmp\_16f1dd96af646d6ec01c5ec51c0fdfafd8206c0a.flv
tmp\_1b1badb376661cc4822989e45505f66e23cb65e4.flv
tmp\_1ca9e47028d5d38e082ff7fbac68fb24d4bafc4f.mp4
tmp\_20d29f3a064272395f59c40cc0c29e7f6c4ff48c.flv
tmp\_23b0c2b21d1471204cd3e3a1de22e8d98849abfe.flv
tmp\_2a3a855c1a79ccfda28c57c4a2bcdaa85c725787.flv
tmp\_2cd2f88502bf8d74a3da0f257b47da823e0d42a0.flv
tmp\_2fe7b38b1cbdfdf5d6075ee2b2cd9fee2cf7d7c1.flv


tmp\_97f5900df59435bc1230615987e24858e9a37f67.flv
tmp\_9b15a52652373497a10b8915b8cc025967962b72.flv
tmp\_9c5fde27983574ca7840da09d40c40bd20820957.flv
tmp\_9f7862bcee961fdec039a3f287ad7c8351cfbed9.flv
tmp\_a0fc0a97772388012c3d2062b36086d2cfb2bc82.flv
tmp\_a15a43f0cf9ce7eab781706a82ee0027d53aae4f.flv
tmp\_a86c9e954e46fae7936a0e70a2ecc1914a2f43e1.flv
tmp\_a89a3193db3354c059dfe4effac05c4667f9c239.flv
tmp\_a8ede4918fb583e92125f6ead7e7cdf6f331084d.flv
tmp\_b053607f9eae018c3916d3fdb0ccb1dc49777456.flv
tmp\_c1025681a6dfd43843735c07e9ec8ac2f08d80fc.flv
tmp\_ccc879ecfb35a1a77667dd8357b71a930c19092c.flv
tmp\_ee417a6b882853ffcd3f78b380b0205a9411f4d6.flv
tmp\_ffe9ad0e2519d04939e04ea128ee565fc3395e5b.flv
tmp\_0ab11b52561e9255423b01f29f7904a6dcadd87b.flv
tmp\_0c06d64856ceae3c5146fd0cae9ac6fc5107083e.flv
tmp\_0dbcff0b6d671e1579ad468eed54d84f7e9b7289.mp4
tmp\_0ed02c5bbb500542f88f0d7285e56e1186ac9bf5.flv
tmp\_13f06a567cbee6f40952cc13817a7774caa72903.flv
tmp\_1e04e15f77363ebb7cdfb8811cd3d5d6d8b18305.flv


,feature,ts,id
0,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
1,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",1.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
2,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",2.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
3,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",3.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
4,"[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",4.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4


In [89]:
query_df.to_csv(DATA_DIR+'/vcdb_rhash_query.csv', index=False)